<a href="https://colab.research.google.com/github/Agniva2006/Face-Recognition-Multi-Class-Classification-/blob/main/Task_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Face Recognition Project

## 1.Google Drive Permission Check

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2.Unziping the dataset

In [ ]:
# Unzip dataset
import zipfile
zip_path = "/content/drive/MyDrive/Comys_Hackathon5.zip"
extract_path = "/content/Comys_Hackathon5"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


## 3.download the libraries

In [ ]:
!pip install facenet-pytorch
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2

## 4.import the libraries

In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
from facenet_pytorch import InceptionResnetV1
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 5.device setting with Facenet model

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# FaceNet Model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

## 6.Image Preprocessing by transformation

In [ ]:
# Image transformation
def preprocess(img_path):
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    img = Image.open(img_path).convert('RGB')
    return transform(img).unsqueeze(0).to(device)

## 7.Embedding fuctions

In [ ]:
def get_embedding(img_path):
    with torch.no_grad():
        img = preprocess(img_path)
        embedding = model(img)
    return embedding.cpu().numpy().flatten()

## 8.Build Reference Embeddings for All Persons (Train + Val folders)


In [ ]:

def build_reference_embeddings(root_paths):
    reference_embeddings = {}
    for root_path in root_paths:
        persons = [p for p in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, p))]
        for person in persons:
            person_folder = os.path.join(root_path, person)
            embeddings = []
            for root, _, files in os.walk(person_folder):
                for file in files:
                    if file.lower().endswith(('jpg', 'jpeg', 'png')):
                        emb = get_embedding(os.path.join(root, file))
                        embeddings.append(emb)
            if embeddings:
                if person in reference_embeddings:
                    reference_embeddings[person].extend(embeddings)
                else:
                    reference_embeddings[person] = embeddings
    print(f"Built embeddings for {len(reference_embeddings)} persons from all folders.")
    return reference_embeddings

## 9.Prediction Logic (Folder-wise Matching Criterion)

In [ ]:
# Predict if test image matches target person folder
def predict_match(test_img_path, target_person, reference_embeddings, threshold=0.6):
    test_emb = get_embedding(test_img_path)
    target_embeds = reference_embeddings.get(target_person)
    if target_embeds is None:
        return 0, 0.0
    sims = cosine_similarity([test_emb], target_embeds)
    max_sim = np.max(sims)
    label = 1 if max_sim >= threshold else 0
    return label, max_sim


## 10.Paths

In [ ]:

train_path = "/content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train"
val_path = "/content/Comys_Hackathon5/Comys_Hackathon5/Task_B/val"

## 11.Build Embeddings with train and validation dataset

In [ ]:

reference_embeddings = build_reference_embeddings([train_path, val_path])

Built embeddings for 1127 persons from all folders.


## 12.Validation Evalution

In [ ]:

correct, total = 0, 0
for person in os.listdir(val_path):
    person_folder = os.path.join(val_path, person)
    if not os.path.isdir(person_folder):
        continue
    for root, _, files in os.walk(person_folder):
        for file in files:
            if file.lower().endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                pred_label, sim = predict_match(img_path, person, reference_embeddings, threshold=0.6)
                correct += int(pred_label == 1)
                total += 1
                print(f"Image: {file}, Actual Person: {person}, Max Similarity: {sim:.4f}, Predicted Label: {pred_label}, Correct: {pred_label == 1}")

print(f"Final Validation Accuracy (Folder Matching Criterion): {correct / total:.4f} ({correct}/{total})")



Image: Chris_Columbus_0001.jpg, Actual Person: Chris_Columbus, Max Similarity: 1.0000, Predicted Label: 1, Correct: True
Image: Chris_Columbus_0001_sunny.jpg, Actual Person: Chris_Columbus, Max Similarity: 1.0000, Predicted Label: 1, Correct: True
Image: Chris_Columbus_0001_foggy.jpg, Actual Person: Chris_Columbus, Max Similarity: 1.0000, Predicted Label: 1, Correct: True
Image: Chris_Columbus_0001_noisy.jpg, Actual Person: Chris_Columbus, Max Similarity: 1.0000, Predicted Label: 1, Correct: True
Image: Chris_Columbus_0001_resized.jpg, Actual Person: Chris_Columbus, Max Similarity: 1.0000, Predicted Label: 1, Correct: True
Image: Chris_Columbus_0001_blurred.jpg, Actual Person: Chris_Columbus, Max Similarity: 1.0000, Predicted Label: 1, Correct: True
Image: Chris_Columbus_0001_rainy.jpg, Actual Person: Chris_Columbus, Max Similarity: 1.0000, Predicted Label: 1, Correct: True
Image: Chris_Columbus_0001_lowlight.jpg, Actual Person: Chris_Columbus, Max Similarity: 1.0000, Predicted Label: 

## 13. dataset details

In [ ]:
print("Train persons:", len(reference_embeddings.keys()))
print("Val persons:", len(os.listdir(val_path)))
print("Overlap persons between train and val:", len(set(reference_embeddings.keys()) & set(os.listdir(val_path))))

Train persons: 1127
Val persons: 250
Overlap persons between train and val: 250


## 14. testing on an random distorted image on train dataset


In [ ]:

from google.colab import files

# Upload a single image file
uploaded = files.upload()

for file_name in uploaded.keys():
    img_path = file_name
    print(f"\nTesting image: {img_path}")

    max_sim = -1
    predicted_person = None

    test_emb = get_embedding(img_path)

    for person, embeds in reference_embeddings.items():
        sims = cosine_similarity([test_emb], embeds)
        person_max_sim = np.max(sims)
        if person_max_sim > max_sim:
            max_sim = person_max_sim
            predicted_person = person

    label = 1 if max_sim >= 0.6 else 0

    print(f"\nPredicted Person: {predicted_person}")
    print(f"Max Cosine Similarity: {max_sim:.4f}")
    print(f"Final Label (1=Match, 0=Non-Match): {label}")


Saving mbappe.png to mbappe.png

Testing image: mbappe.png

Predicted Person: 082_frontal
Max Cosine Similarity: 0.5976
Final Label (1=Match, 0=Non-Match): 0


In [ ]:
# Print all layer names, weights and biases in FaceNet
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name} | shape: {param.shape}")


conv2d_1a.conv.weight | shape: torch.Size([32, 3, 3, 3])
conv2d_1a.bn.weight | shape: torch.Size([32])
conv2d_1a.bn.bias | shape: torch.Size([32])
conv2d_2a.conv.weight | shape: torch.Size([32, 32, 3, 3])
conv2d_2a.bn.weight | shape: torch.Size([32])
conv2d_2a.bn.bias | shape: torch.Size([32])
conv2d_2b.conv.weight | shape: torch.Size([64, 32, 3, 3])
conv2d_2b.bn.weight | shape: torch.Size([64])
conv2d_2b.bn.bias | shape: torch.Size([64])
conv2d_3b.conv.weight | shape: torch.Size([80, 64, 1, 1])
conv2d_3b.bn.weight | shape: torch.Size([80])
conv2d_3b.bn.bias | shape: torch.Size([80])
conv2d_4a.conv.weight | shape: torch.Size([192, 80, 3, 3])
conv2d_4a.bn.weight | shape: torch.Size([192])
conv2d_4a.bn.bias | shape: torch.Size([192])
conv2d_4b.conv.weight | shape: torch.Size([256, 192, 3, 3])
conv2d_4b.bn.weight | shape: torch.Size([256])
conv2d_4b.bn.bias | shape: torch.Size([256])
repeat_1.0.branch0.conv.weight | shape: torch.Size([32, 256, 1, 1])
repeat_1.0.branch0.bn.weight | shape: 